In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import pyswarms as ps
import numpy as np
import pandas as pd
import os

In [2]:
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
# Define o tamanho da imagem e os tipos de rua
im_size = 350  # Exemplo de tamanho de imagem


street_types = ['Apple', 'Banana', 'Cocos']
path = '../DatasetFruits/'
path_test = '../DatasetFruits_Test/'

"""
street_types = ['clean', 'litter', 'recycle']
path = '../Dataset/'
path_test = '../Dataset_Test/'
"""

"\nstreet_types = ['clean', 'litter', 'recycle']\npath = '../Dataset/'\npath_test = '../Dataset_Test/'\n"

In [4]:
# Criando dados de treino, validação e teste
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

# Contando as ocorrências de cada tipo de rua e imprimindo
streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: Apple     429
Banana    427
Cocos     427
dtype: int64


In [5]:
# Convertendo etiquetas para valores numéricos
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Embaralhando e dividindo os dados de treino e validação
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=415)

# Imprimindo as formas dos conjuntos de dados
print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (1026, 350, 350, 3)
Validation shape: (257, 350, 350, 3)
Test shape: (189, 350, 350, 3)


In [6]:
# Definindo o modelo da rede neural convolucional
model = Sequential([
    # Primeira camada convolucional
    Conv2D(64, (3, 3), activation='relu', input_shape=(im_size, im_size, 3)), #32
    MaxPooling2D((2, 2)),
    
    # Segunda camada convolucional
    #Conv2D(64, (3, 3), activation='relu'),
    #MaxPooling2D((2, 2)),
    
    # Terceira camada convolucional
    #Conv2D(128, (3, 3), activation='relu'),
    #MaxPooling2D((2, 2)),

    # Camada de achatamento para converter as características 3D em 1D
    Flatten(),
    
    # Camada densa com regularização para reduzir o sobreajuste
    #Dense(512, activation='relu'),
    Dropout(0.5),

    # Camada de saída com softmax para classificação multiclasse
    Dense(len(street_types), activation='softmax')   
])

# Compilando o modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',  # Função de perda para classificação multiclasse
              metrics=['accuracy'])  # Métrica de avaliação

# Treinando o modelo com os dados de treino e validando com o conjunto de validação
model.fit(train_x, 
          train_y, 
          epochs=5,  # Número de épocas de treinamento
          #batch_size=64,  # Tamanho do lote, descomente e ajuste se necessário
          validation_data=(val_x, val_y))  # Usando os dados de validação aqui

Epoch 1/5
33/33 [==============================] - 44s 1s/step - loss: 3.6098 - accuracy: 0.9152 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/5
33/33 [==============================] - 38s 1s/step - loss: 1.7428e-09 - accuracy: 1.0000 - val_loss: 2.3192e-09 - val_accuracy: 1.0000
Epoch 3/5
33/33 [==============================] - 37s 1s/step - loss: 4.6475e-09 - accuracy: 1.0000 - val_loss: 4.6385e-09 - val_accuracy: 1.0000
Epoch 4/5
33/33 [==============================] - 38s 1s/step - loss: 9.5274e-09 - accuracy: 1.0000 - val_loss: 4.6385e-09 - val_accuracy: 1.0000
Epoch 5/5
33/33 [==============================] - 39s 1s/step - loss: 7.7846e-09 - accuracy: 1.0000 - val_loss: 4.6385e-09 - val_accuracy: 1.0000


In [7]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels_encoded, verbose=1);

y_pred_test = model.predict(test_images);
y_pred_test_classes = np.argmax(y_pred_test, axis=1);

y_true = test_labels_encoded;
y_true_binarized = label_binarize(test_labels_encoded, classes=np.unique(test_labels_encoded));

6/6 [==============================] - 1s 189ms/step


In [8]:
# Avaliação do modelo nos dados de teste
print(f"Test accuracy: {test_accuracy*100:.2f}%" + "\n");

# Matriz de confusão
confusion_mtx = confusion_matrix(y_true, y_pred_test_classes)
print("Confusion Matrix:")
print(confusion_mtx)

# Relatório de classificação
class_report = classification_report(y_true, y_pred_test_classes, target_names=label_encoder.classes_)
print("\nClassification Report:")
print(class_report)
print("\n")

y_true_binarized = label_binarize(test_labels_encoded, classes=np.unique(test_labels_encoded));

# Calculando o AUC para cada classe
for i in range(y_true_binarized.shape[1]):  # Percorre cada classe
    auc_score = roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i])
    print(f'AUC for class {label_encoder.classes_[i]}: {auc_score:.2f}')

Test accuracy: 100.00%

Confusion Matrix:
[[63  0  0]
 [ 0 63  0]
 [ 0  0 63]]

Classification Report:
              precision    recall  f1-score   support

       Apple       1.00      1.00      1.00        63
      Banana       1.00      1.00      1.00        63
       Cocos       1.00      1.00      1.00        63

    accuracy                           1.00       189
   macro avg       1.00      1.00      1.00       189
weighted avg       1.00      1.00      1.00       189



AUC for class Apple: 1.00
AUC for class Banana: 1.00
AUC for class Cocos: 1.00
